In [15]:
import time
import datetime
import os
from Research.backtest.bt import BTDaily
import matplotlib.pyplot as plt
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp
import sys
sys.path.append("/home/wuwenjun/jupyter_code/Shannon/AlphaNet/packages/")
import AlphaNet
from AlphaNet.Data import concat_original_data , generate_shift_data

In [16]:
task_info = np.load("/home/ShareFolder/feature_platform/ti0/wuwenjun/#Factor_Description/Task.npy",allow_pickle=True).item()
task_info["Task"]

,Alpha_Name,sequence,LR,epoch_num,feature_num,start_date,end_date,status,description
task_id,,,,,,,,,
0,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-01-01,2019-06-01,finished,None
1,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2019-06-01,2020-01-01,finished,None
2,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-01-01,2020-06-01,finished,None
3,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2020-06-01,2021-01-01,finished,None
4,Original_Input_Moving_10_Shift_5,5,"[0.001, 0.0001]","[30, 20]",108,2021-01-01,2021-06-01,finished,None
...,...,...,...,...,...,...,...,...,...
195,AlphaNet_8Input_corr_filter_Shift_1_Sequence_2...,20,"[0.001, 1e-05]","[30, 30]",54,2019-01-01,2019-06-01,Finish: 19:31_01:39 | 01-31,"{'target': '10dRet', 'alpha_name': 'AlphaNet_8..."
196,AlphaNet_8Input_corr_filter_Shift_1_Sequence_2...,20,"[0.001, 1e-05]","[30, 30]",54,2019-06-01,2020-01-01,Finish: 19:32_02:24 | 01-31,"{'target': '10dRet', 'alpha_name': 'AlphaNet_8..."
197,AlphaNet_8Input_corr_filter_Shift_1_Sequence_2...,20,"[0.001, 1e-05]","[30, 30]",54,2020-01-01,2020-06-01,Finish: 19:33_02:57 | 01-31,"{'target': '10dRet', 'alpha_name': 'AlphaNet_8..."


In [18]:
alpha_list = [i.replace("10","5") for i in alpha_list10]
alpha_list

['COV_div_open_close_div_high_close_5',
 'COV_div_open_close_div_low_close_5',
 'COV_div_open_close_close_5',
 'COV_div_open_close_volume_sqrt_5',
 'COV_div_open_close_pct_chg_5',
 'COV_div_high_close_div_low_close_5',
 'COV_div_high_close_close_5',
 'COV_div_high_close_volume_sqrt_5',
 'COV_div_high_close_pct_chg_5',
 'COV_div_low_close_close_5',
 'COV_div_low_close_volume_sqrt_5',
 'COV_close_div_vwap_close_5',
 'COV_close_volume_sqrt_5',
 'COV_close_pct_chg_5',
 'COV_div_vwap_close_volume_sqrt_5',
 'COV_volume_sqrt_pct_chg_5',
 'COV_volume_sqrt_turnover_5',
 'CORR_div_open_close_div_high_close_5',
 'CORR_div_open_close_div_low_close_5',
 'CORR_div_open_close_close_5',
 'CORR_div_open_close_div_vwap_close_5',
 'CORR_div_open_close_pct_chg_5',
 'CORR_div_high_close_div_low_close_5',
 'CORR_div_high_close_close_5',
 'CORR_div_high_close_div_vwap_close_5',
 'CORR_div_high_close_pct_chg_5',
 'CORR_div_low_close_close_5',
 'CORR_div_low_close_div_vwap_close_5',
 'CORR_div_low_close_volume

In [19]:
target = None
alpha_name = "AlphaNet_8Input_corr_filter_5dMoving"
shift = 1
sequence = 20
LR = [0.001,0.00001,0.0000001]
epoch_num = [30,30,30]

In [20]:
# read_task
output_path = "/home/wuwenjun/Data/"
concat_original_data(alpha_name=alpha_name,alpha_list=alpha_list,output_path=output_path)
generate_shift_data(alpha_name=alpha_name,shift=shift,sequence=sequence,target=target,data_path=output_path)

['COV_div_open_close_div_high_close_5', 'COV_div_open_close_div_low_close_5', 'COV_div_open_close_close_5', 'COV_div_open_close_volume_sqrt_5', 'COV_div_open_close_pct_chg_5', 'COV_div_high_close_div_low_close_5', 'COV_div_high_close_close_5', 'COV_div_high_close_volume_sqrt_5', 'COV_div_high_close_pct_chg_5', 'COV_div_low_close_close_5', 'COV_div_low_close_volume_sqrt_5', 'COV_close_div_vwap_close_5', 'COV_close_volume_sqrt_5', 'COV_close_pct_chg_5', 'COV_div_vwap_close_volume_sqrt_5', 'COV_volume_sqrt_pct_chg_5', 'COV_volume_sqrt_turnover_5', 'CORR_div_open_close_div_high_close_5', 'CORR_div_open_close_div_low_close_5', 'CORR_div_open_close_close_5', 'CORR_div_open_close_div_vwap_close_5', 'CORR_div_open_close_pct_chg_5', 'CORR_div_high_close_div_low_close_5', 'CORR_div_high_close_close_5', 'CORR_div_high_close_div_vwap_close_5', 'CORR_div_high_close_pct_chg_5', 'CORR_div_low_close_close_5', 'CORR_div_low_close_div_vwap_close_5', 'CORR_div_low_close_volume_sqrt_5', 'CORR_div_low_clos

100%|██████████| 1705/1705 [00:18<00:00, 91.68it/s] 


[19, 18, 17, 16, 15, 14, 13, 12, 11, 10, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]


100%|██████████| 1291/1291 [10:54<00:00,  1.97it/s]


,,COV_div_open_close_div_high_close_5_Shift_19,COV_div_open_close_div_low_close_5_Shift_19,COV_div_open_close_close_5_Shift_19,COV_div_open_close_volume_sqrt_5_Shift_19,COV_div_open_close_pct_chg_5_Shift_19,COV_div_high_close_div_low_close_5_Shift_19,COV_div_high_close_close_5_Shift_19,COV_div_high_close_volume_sqrt_5_Shift_19,COV_div_high_close_pct_chg_5_Shift_19,COV_div_low_close_close_5_Shift_19,...,ZSCORE_pct_chg_5_Shift_0,RETURN_div_open_close_5_Shift_0,RETURN_div_high_close_5_Shift_0,RETURN_div_low_close_5_Shift_0,RETURN_div_vwap_close_5_Shift_0,RETURN_volume_sqrt_5_Shift_0,RETURN_pct_chg_5_Shift_0,DECAY_div_high_close_5_Shift_0,DECAY_div_low_close_5_Shift_0,DECAY_turnover_5_Shift_0
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2016-02-05 09:30:00,000001,0.000743,0.000183,-0.328205,-2.608878,-0.105278,0.000119,-0.200347,-1.630504,-0.081003,-0.065746,...,0.264911,-1.449477e+00,-0.756533,-29074050.0,-1.177049e+00,0.110505,2.938300e+10,0.007757,-0.010812,0.307867
2016-02-15 09:30:00,000001,0.000333,0.000148,-0.525332,-2.644009,-0.061182,0.000088,-0.351596,-1.704566,-0.043565,-0.129569,...,-0.105246,3.003436e+08,-0.369959,299940288.0,8.854202e+07,-0.294616,-1.094242e+00,0.006333,-0.006523,0.272600
2016-02-16 09:30:00,000001,0.000328,0.000101,-0.311013,-2.194751,-0.055421,0.000084,-0.246143,-1.644112,-0.045431,-0.043436,...,-0.011446,-1.722960e+00,-0.713994,-81784408.0,-1.680929e+00,-0.183487,6.895000e+09,0.005470,-0.008665,0.254913
2016-02-17 09:30:00,000001,0.000206,0.000092,-0.000827,-2.177916,-0.040362,0.000075,-0.004179,-1.606188,-0.034064,-0.071312,...,0.086949,-1.907767e+07,-0.751499,-8966462.0,-1.176964e+01,0.077317,4.681824e-01,0.004249,-0.011489,0.287427
2016-02-18 09:30:00,000001,0.000153,0.000134,-0.008488,-0.239755,-0.018324,0.000087,-0.019153,-0.034048,-0.015416,-0.062096,...,0.431581,-1.182347e+08,0.698272,-76422280.0,-4.050801e+07,0.459862,2.403600e+10,0.005065,-0.013514,0.360860
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-05-25 09:30:00,688981,0.000110,0.000097,-0.007377,0.148062,-0.020993,0.000067,-0.003886,0.109057,-0.013373,-0.005070,...,0.020728,-3.090695e+00,-0.962156,-230359648.0,-1.963511e+00,-0.106356,3.356700e+10,0.007511,-0.009985,1.222607
2021-05-26 09:30:00,688981,0.000107,0.000102,-0.008025,0.079445,-0.020557,0.000062,-0.003091,0.175850,-0.014002,-0.005763,...,0.184022,-2.336150e+00,-0.505130,-20665450.0,3.566052e+06,0.312623,9.434000e+09,0.005611,-0.010964,1.468233
2021-05-27 09:30:00,688981,0.000078,0.000028,-0.001148,0.330677,-0.011548,0.000035,-0.001694,0.483292,-0.010174,-0.000070,...,0.106757,4.372247e+01,1.084625,36514268.0,4.750664e+00,0.323878,-4.102000e+09,0.008434,-0.008146,1.574267
